In [ ]:
import pandas, numpy as np,re
import tqdm
import collections
import xml, itertools, re, xml.etree.ElementTree as ET
from html.parser import HTMLParser
floatParse = '[0-9]*[\.,]?[0-9]+'

import datetime, tqdm
pars =HTMLParser()
def findInXML(s, et): 
    if isinstance(et, str):
        raise ValueError('et should be a tree, not a string')
    r =et.find('.//row[@NombreCampo="%s"]' % s)
    if r is not None:
        return r.get('ValorCampo')
    return None
def prettyPrintXML(s):
    r = xml.dom.minidom.parseString(s) #r.RegistroXML)
    print(pars.unescape(r.toprettyxml()))
    
import re
# Maybe I should start tokenizing
cleanWhites = re.compile("[^\S\n]+")
import unicodedata
def remove_diacritics(text):
    """
    Returns a string with all diacritics (aka non-spacing marks) removed.
    For example "Héllô" will become "Hello".
    Useful for comparing strings in an accent-insensitive fashion.
    """
    normalized = unicodedata.normalize("NFKD", text)
    return "".join(c for c in normalized if unicodedata.category(c) != "Mn")

def cleanString(text, removeChars = '-:,;', removeWords = []):
    for c in removeChars:
        text = re.sub('(?<![0-9])\%s' %c, ' ',  text)
        text = re.sub('\%s(?![0-9])' %c, ' ',  text)

    text = cleanWhites.sub(' ', text)
    for w in removeWords:
        text = text.replace(' ' + w + ' ', ' ')
    return text.strip()

# Read
- FC 
- FR
- T

In [ ]:
casos = pandas.read_csv('casos2019/casos.csv', index_col = 0)
pacientes = pandas.read_csv('casos2019/pacientes.csv', index_col = 0)
registros = pandas.read_csv('casos2019/registros.csv', index_col = 0)
diagnosis = pandas.read_csv('Venezolanas/diagnosis.csv', index_col = 0)
procedimientos = pandas.read_csv('casos2019/procedimientos.csv', index_col = 0)
procedimientosDesc = pandas.read_csv('Venezolanas/procedimientosID.csv', index_col = 0)
medidas = pandas.read_csv('casos2019/enfermeriaMedidas.csv', index_col = 0)

In [ ]:
registrosByCasos = registros.groupby('Caso')

In [ ]:
def identifyPressureInText(text):
    text = cleanString(text).upper()
    return re.findall('(?:TA|PA|TENSION|PRESION)\s*([0-9]+)[-/]([0-9]+)\s*MMHG', text )
    
def identifyHRInText(text):
    text = cleanString(text).upper()
    return re.findall('(?:FC)\s*([0-9]+)', text )

def identifyRRInText(text):
    text = cleanString(text).upper()
    return re.findall('(?:FR)\s*([0-9]+)', text )

def identifyTInText(text):
    text = cleanString(text).upper()
    return re.findall('(?:T)\s*(%s)' % floatParse, text )


In [ ]:
for c in registrosByCasos.groups.keys():
    if c.startswith('TRI'):
        continue
    print(c)

In [ ]:
def isMaternal(c, registers, raiseError = False):
    """
    Gets whether the record is of the mother or the newborn
    
    TODO: double check
    """
    asunto = remove_diacritics(str(c.Asunto)).lower()
    try:
        if 'neonato' in asunto or 'nacido' in asunto: 
            return  False
        elif 'Registro del recién nacido' == registros.loc[int(c.Padre)].Asunto:
            return False
        else:
            return True
    except KeyError as e:
        if raiseError:
            raise(e)
        else:
            return True

In [ ]:
def identifyPressureInText(text):
    text = cleanString(text).upper()
    return re.findall('(?:TA|PA|TENSION|PRESION)\s*([0-9]+)[-/]([0-9]+)\s*MMHG', text )
    
def identifyHRInText(text):
    text = cleanString(text).upper()
    return re.findall('(?:FC)\s*([0-9]+)', text )

def identifyRRInText(text):
    text = cleanString(text).upper()
    return re.findall('(?:FR)\s*([0-9]+)', text )

def identifyTInText(text):
    text = cleanString(text).upper()
    return re.findall('(?:T)\s*(%s)' % floatParse, text )


def parseVitalSignsFromRegister(r):
    res = {}
    et = ET.fromstring(r.RegistroXML)
    fecha = r.FechaAsignacionRegistro
    txt = fullCleanTxt(findInXML('DescripcionNota', et))

    # Pressure
    pressure = findInXML('Presion', et)
    if not pressure or pressure == '/':
        if txt:
            pressure = identifyPressureInText(txt)
            if pressure:
                pressure = '/'.join(pressure[0])
    if pressure:
        res['P'[ = (fecha, pressure)

    # FC
    fc = findInXML('FrecuenciaCardiaca', et)
    if not fc:
        if txt:
            fc = identifyHRInText(txt)
            if fc:
                fc = fc[0]
    if fc:
        res['hr'] = (fecha, fc)

    # RR
    rr = findInXML('FrecuenciaRespiratoria', et)
    if not rr:
        if txt:
            rr = identifyRRInText(txt)
            if rr:
                rr = rr[0]
    if rr:
        res['rr'] = (fecha, fc)

    # T    
    temperature = findInXML('Temperatura', et)
    if not temperature or temperature == '/':
        if txt:
            temperature = identifyTInText(txt)
            if temperature:
                temperature = temperature[0]
    if temperature:
        res['T'] = (fecha, temperatureture)
    return res


In [ ]:
c = 'AD348292'
allPressuresAllCases = collections.defaultdict(list)
allHRAllCases = collections.defaultdict(list)
allRRAllCases = collections.defaultdict(list)
allTAllCases = collections.defaultdict(list)

for c in tqdm.tqdm_notebook(registrosByCasos.groups.keys()):
    if c.startswith('TRI'):
        continue
    allPressures = []
    allTs = []
    allRR = []
    allHR = []

    for _, row in registrosByCasos.get_group(c).iterrows():
        #Check whether report is of newborn or of the mother
        if not isMaternal(row, registros):
            continue
        r = row.RegistroXML
        et = ET.fromstring(r)
        fecha = row.FechaAsignacionRegistro
        txt = remove_diacritics(findInXML('DescripcionNota', et))

        # Pressure
        pressure = findInXML('Presion', et)
        if not pressure or pressure == '/':
            if txt:
                pressure = identifyPressureInText(txt)
                if pressure:
                    pressure = '/'.join(pressure[0])
        if pressure:
            allPressures.append([fecha, pressure])
            
        # FC
        fc = findInXML('FrecuenciaCardiaca', et)
        if not fc:
            if txt:
                fc = identifyHRInText(txt)
                if fc:
                    fc = fc[0]
        if fc:
            allHR.append([fecha, fc])

        # RR
        rr = findInXML('FrecuenciaRespiratoria', et)
        if not rr:
            if txt:
                rr = identifyRRInText(txt)
                if rr:
                    rr = rr[0]
        if rr:
            allRR.append([fecha, rr])

        # T    
        temperature = findInXML('Temperatura', et)
        if not temperature or temperature == '/':
            if txt:
                temperature = identifyTInText(txt)
                if temperature:
                    temperature = temperature[0]
        if temperature:
            allTs.append([fecha, temperature])

    allPressuresAllCases[c] = allPressures
    allTAllCases[c] = allTs
    allHRAllCases[c] = allHR    
    allRRAllCases[c] = allRR

In [ ]:
for p in allHRAllCases:
    if len(allRRAllCases[c]) != len(allPressuresAllCases[c]):
        print(p)

In [ ]:
medidasGrouped = medidas.groupby(['IdAdmision', 'Fecha'])


In [ ]:
for (c, date) in tqdm.tqdm_notebook(medidasGrouped.groups.keys()):
    if date != date:
        continue
    m = medidasGrouped.get_group((c, date))
    if any(m.CodSignoVitalTipo == 'PRSI') and  any(m.CodSignoVitalTipo == 'PRDI'):
        tas = m[m.CodSignoVitalTipo == 'PRSI'].iloc[0].Valor
        tad = m[m.CodSignoVitalTipo == 'PRDI'].iloc[0].Valor
    elif any(m.CodSignoVitalTipo == 'PAS') and  any(m.CodSignoVitalTipo == 'PAS'):
        tas = m[m.CodSignoVitalTipo == 'PAS'].iloc[0].Valor
        tad = m[m.CodSignoVitalTipo == 'PAD'].iloc[0].Valor
    else:
        tas, tad = None, None
    if tas and tad:
        allPressuresAllCases[c].append([date, '%d/%d' % (tas, tad)])

    # FC
    if any(m.CodSignoVitalTipo == 'FRCA'):
        fc = m[m.CodSignoVitalTipo == 'FRCA'].iloc[0].Valor
    elif any(m.CodSignoVitalTipo == 'FC'):
        fc = m[m.CodSignoVitalTipo == 'FC'].iloc[0].Valor
    else:
        fc = None
    
    if fc:
        allHRAllCases[c].append([date, '%d' % (fc)])
        
    # RR
    if any(m.CodSignoVitalTipo == 'FRRE'):
        rr = m[m.CodSignoVitalTipo == 'FRRE'].iloc[0].Valor
    elif any(m.CodSignoVitalTipo == 'FR'):
        rr = m[m.CodSignoVitalTipo == 'FR'].iloc[0].Valor
    else:
        rr = None
        
    if rr:
        allRRAllCases[c].append([date, rr])

    # T
    if any(m.CodSignoVitalTipo == 'T'):
        t = m[m.CodSignoVitalTipo == 'T'].iloc[0].Valor
    else:
        t = None
    if t:
        allTAllCases[c].append([date, '%f' % t])


In [ ]:
class MeasurementsByDay():
    def __init__(self, name = 'pressure'):
        self.maxVals = collections.defaultdict(lambda: 0) 
        self.minVals = collections.defaultdict(lambda: 1000) 
        self.name = name
    def addMeasurement(self, date, p):
        date =  datetime.datetime.strptime(date.split()[0], '%Y-%m-%d')
        if  not isinstance(p, str) or '/' not in p:
            p = '%s/%s' % (p, p)
        maxVal, minVal = p.split('/')
        maxVal, minVal = float(maxVal.replace(',','')), float(minVal.replace(',',''))
        if maxVal and minVal:
            self.maxVals[date] = max(self.maxVals[date], maxVal)
            self.minVals[date] = min(self.minVals[date], minVal)
    def toDict(self):
        res = {}
        for i, d in enumerate(sorted(self.maxVals.keys())):
            res['day_%03d_%s' % (i, self.name)] = str(self.maxVals[d]) + '/' +  str(self.minVals[d])
            res['day_%03d' % i] = d

        return res

In [ ]:
import datetime
measurements = {}
for c, v in tqdm.tqdm_notebook(allPressuresAllCases.items()):
    pdP = MeasurementsByDay('Presion')
    pdT = MeasurementsByDay('Temperatura')
    pdHR = MeasurementsByDay('Frec. Cardi.')
    pdRR = MeasurementsByDay('Frec. resp.')
    for (d, p) in v:
        if p == '/':
            continue
        pdP.addMeasurement(d, p)
        
    for (d, p) in allTAllCases[c]:
        pdT.addMeasurement(d, p)
        
    for (d, p) in allHRAllCases[c]:
        pdHR.addMeasurement(d, p)
        
    for (d, p) in allRRAllCases[c]:
        pdRR.addMeasurement(d, p)

    measurements[c] = pdP.toDict()
    measurements[c].update(pdT.toDict())
    measurements[c].update(pdHR.toDict())
    measurements[c].update(pdRR.toDict())


In [ ]:
df = pandas.DataFrame.from_dict({ k:v  for k, v in measurements.items()} , orient = 'index').sort_index(axis = 1)
df.to_csv('maxMinMeasurements.csv')

In [ ]:
import parsingDatabaseUtils, importlib
importlib.reload(parsingDatabaseUtils)
r = registros.loc[1365341]
rET = ET.fromstring(r.RegistroXML)
rTxt = parsingDatabaseUtils.fullCleanTxt(parsingDatabaseUtils.findInXML('AntecedentesHTML', rET))
rTxt = parsingDatabaseUtils.removeWords(rTxt, ['de', 'y', 'a', 'el', 'los'])

print(rTxt)

In [ ]:
import parsingDatabaseUtils

date =  '\(?' +  '((?:[0-9]+)'+ sep + separadorFecha + sep + '(?:[0-9]+|%s)'%  '|'.join(meses) + \
                       sep + separadorFecha + sep + '(?:[0-9]+))' + '\)?' 

# read paraclinics. Assume that the date is just before them, or in the same line

pos = ['pos', '\+', 'reac']
neg = ['neg', '-', 'no']
vih1  = 'vih\s*[1i]'
vih2 = 'vih\s*(:?2|ii)'
vih12 = 'vih\s*[1i]\s*(:?2|ii)'
prt = '(:?PRUEBA RAPIDA TREPONEMICA|PRT)'
vdrl = '(:?VDRL)'

hematology = {
'HCT' : '(HCT|HTO|HEMATO[A-Z]*)',
'HB' : '(HB|HEMOGLOB[A-Z]*)',
'LEU' : '(LEU[A-Z]*)',
'NEU' : '(NEU[A-Z]*)',
'LIN' : '(LIN[A-Z]*)',
'MONO' : '(MONO[A-Z]*)',
'PLAQ' : '(PL[A-Z]*)'}

In [ ]:
date =  '\(?' +  '((?:[0-9]+)'+ sep + separadorFecha + sep + '(?:[0-9]+|%s)'%  '|'.join(meses) + \
                       sep + separadorFecha + sep + '(?:[0-9]+))' + '\)?' 

# read paraclinics. Assume that the date is just before them, or in the same line

pos = ['pos', '\+', 'reac']
neg = ['neg', '-', 'no']
vih1  = 'vih\s*[1i]'
vih2 = 'vih\s*(:?2|ii)'
vih12 = 'vih\s*[1i]\s*(:?2|ii)'
prt = '(:?PRUEBA RAPIDA TREPONEMICA|PRT)'
vdrl = '(:?VDRL)'

hematology = {
'HCT' : '(HCT|HTO|HEMATO[A-Z]*)',
'HB' : '(HB|HEMOGLOB[A-Z]*)',
'LEU' : '(LEU[A-Z]*)',
'NEU' : '(NEU[A-Z]*)',
'LIN' : '(LIN[A-Z]*)',
'MONO' : '(MONO[A-Z]*)',
'PLAQ' : '(PL[A-Z]*)'}

meses = ['ENERO', 'FEBRERO', 'MARZO', 'ABRIL', 'MAYO', 'JUNIO', 'JULIO', 'AGOSTO', 'SEPTIEMBRE', 'OCTUBRE', 'NOVIEMBRE', 'DICIEMBRE']
meses = meses + list(map(lambda s: s[:3], meses))
sep= '\s*[,;:]?\s*'
separadorFecha = '(?:[\.\\/-]|DE|DEL|\s)'
date =  '\(?' +  '((?:[0-9]+)'+ sep + separadorFecha + sep + '(?:[0-9]+|%s)'%  '|'.join(meses) + \
                       sep + separadorFecha + sep + '(?:[0-9]+))' + '\)?' 

def parseParaclinics(r):
    if isinstance(r, str):
        rET = ET.fromstring(r.RegistroXML)
        rTxt = parsingDatabaseUtils.fullCleanTxt(parsingDatabaseUtils.findInXML('AntecedentesHTML', rET))
        rTxt = parsingDatabaseUtils.removeWords(rTxt, ['de', 'y', 'a', 'el', 'los'])
    else:
        rTxt = r
    results = collections.defaultdict(dict)
    for i, l in enumerate(rTxt.splitlines()):
        d = re.findall(date, l)
        if len(d) == 1.:
            lastDate = d[0]
        elif len(d):
            lastDate = None
        if len(d) <= 1:
            #Hematology
            for p, v in hematology.items():
                hematologyRes = re.findall('%s (%s)' % (v, floatParse), l, re.IGNORECASE)
                if hematologyRes:
                    results[lastDate][p] = hematologyRes[0][1]

            #sifilis
            sif1 = vdrl + " (%s)" % '|'.join(pos + neg)
            searchSif1 = re.findall(sif1, l, re.IGNORECASE)
            sif2 = prt + " (%s)" % '|'.join(pos + neg)    
            searchSif2 = re.findall(sif2, l, re.IGNORECASE)
            if searchSif1:
                results[lastDate]['vdrl'] =searchSif1[0][1]
            if searchSif2:
                results[lastDate]['prt'] =searchSif2[0][1]
            # TODO: vih
    return results

In [ ]:
for p, m in 

In [ ]:
df